In [1]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [2]:
preprocessor = mz.preprocessors.CDSSMPreprocessor(
    truncated_length_left=10,
    truncated_length_right=10
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 7668.28it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:05<00:00, 3698.44it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 3025320.56it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8849.99it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4454.24it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 806532.71it/s]
Processing text_left with chain_transform of NgramLetter => WordHashing: 100%|██████████| 2118/2118 [00:02<00:00, 818.51it/s]
Processing text_right with chain_tran

In [4]:
preprocessor.context

{'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f97c0a3ae48>,
 'vocab_size': 9645,
 'embedding_input_dim': 9645}

In [5]:
padding_callback = mz.models.CDSSM.get_default_padding_callback()
fixed_padding_callback = mz.dataloader.callbacks.CDSSMPadding(
    fixed_length_left=10,
    fixed_length_right=100
)

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    sort=False,
    resample=True,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    callback=fixed_padding_callback
)

In [7]:
model = mz.models.CDSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['vocab_size']
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['strides'] = 1
model.params['padding'] = 0
model.params['conv_activation_func'] = 'tanh'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

CDSSM(
  (net_left): Sequential(
    (0): Conv1d(9645, 64, kernel_size=(3,), stride=(1,))
    (1): Tanh()
    (2): Dropout(p=0.8)
    (3): AdaptiveMaxPool1d(output_size=1)
    (4): Squeeze()
    (5): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (net_right): Sequential(
    (0): Conv1d(9645, 64, kernel_size=(3,), stride=(1,))
    (1): Tanh()
    (2): Dropout(p=0.8)
    (3): AdaptiveMaxPool1d(output_size=1)
    (4): Squeeze()
    (5): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (out): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
  )
)
Trainable params:

In [8]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [9]:
trainer.run()

[Iter-102 Loss-0.848]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.402 - normalized_discounted_cumulative_gain@5(0.0): 0.4956 - mean_average_precision(0.0): 0.4465



[Iter-204 Loss-0.669]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.427 - normalized_discounted_cumulative_gain@5(0.0): 0.5156 - mean_average_precision(0.0): 0.4666



[Iter-306 Loss-0.519]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4112 - normalized_discounted_cumulative_gain@5(0.0): 0.5005 - mean_average_precision(0.0): 0.4535



[Iter-408 Loss-0.434]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4305 - normalized_discounted_cumulative_gain@5(0.0): 0.5034 - mean_average_precision(0.0): 0.4632



[Iter-510 Loss-0.337]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4273 - normalized_discounted_cumulative_gain@5(0.0): 0.5008 - mean_average_precision(0.0): 0.4635



[Iter-612 Loss-0.292]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4492 - normalized_discounted_cumulative_gain@5(0.0): 0.5217 - mean_average_precision(0.0): 0.4814



[Iter-714 Loss-0.245]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4449 - normalized_discounted_cumulative_gain@5(0.0): 0.5269 - mean_average_precision(0.0): 0.4835



[Iter-816 Loss-0.205]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4507 - normalized_discounted_cumulative_gain@5(0.0): 0.5304 - mean_average_precision(0.0): 0.4787



[Iter-918 Loss-0.191]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4453 - normalized_discounted_cumulative_gain@5(0.0): 0.5247 - mean_average_precision(0.0): 0.4797



[Iter-1020 Loss-0.174]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4506 - normalized_discounted_cumulative_gain@5(0.0): 0.5242 - mean_average_precision(0.0): 0.4783

Cost time: 2333.8692882061005s
